In [3]:
# Cellule 1 — Imports
import numpy as np
import random


In [5]:
# Cellule 2 — Environnement V4 : production en lots, bornes 0-10, commande +5 (fixe)

class WorkshopEnvV4:
    """
    Modélisation 4 :
    - Actions :
        0 = Attendre
        1..10 = Produire k unités de Produit 1
        11..20 = Produire k unités de Produit 2 (k = action-10)
        21 = Commander +5 MP (règle d'origine)
    - Bornes :
        stock_raw <= 10
        stock_sell <= 10
    - Coût de stockage : 0.5 × stock_sell
    - Durées :
        P1 : 1 par unité
        P2 : 3 par unité
        Commander : 1
        Attendre : 1
    """

    def __init__(self, max_steps=50, holding_cost=0.5):
        self.max_steps = max_steps
        self.holding_cost = holding_cost
        self.reset()

    def reset(self):
        self.stock_raw = np.random.randint(0, 11)   # 0-10
        self.stock_sell = 0                         # 0-10
        self.steps = 0
        return (self.stock_raw, self.stock_sell)

    def step(self, action):
        reward = 0.0

        # ACTION 0 — Attendre
        if action == 0:
            reward = -1 if self.stock_raw == 0 else 0
            self.steps += 1

        # ACTIONS 1..10 — Produire k P1
        elif 1 <= action <= 10:
            k = action
            if self.stock_raw >= k:
                self.stock_raw -= k
                self.stock_sell = min(10, self.stock_sell + k)
                reward = 2.0 * k
            self.steps += k * 1

        # ACTIONS 11..20 — Produire k P2
        elif 11 <= action <= 20:
            k = action - 10
            cost_mp = 2 * k
            if self.stock_raw >= cost_mp:
                self.stock_raw -= cost_mp
                self.stock_sell = min(10, self.stock_sell + k)
                reward = 20.0 * k
            self.steps += k * 3

        # ACTION 21 — Commander +5 MP (règle d'origine)
        elif action == 21:
            self.stock_raw = min(10, self.stock_raw + 5)
            reward = -5
            self.steps += 1

        # BORNES FINALES
        self.stock_raw = max(0, min(10, self.stock_raw))
        self.stock_sell = max(0, min(10, self.stock_sell))

        # Coût de stockage
        reward -= self.holding_cost * self.stock_sell

        # Fin d'épisode
        done = self.steps >= self.max_steps
        return (self.stock_raw, self.stock_sell), reward, done, {}


In [7]:
# Cellule 3 — Instanciation de V4 (bornes 0-10 strictes)
env = WorkshopEnvV4(max_steps=50, holding_cost=0.5)


In [9]:
# Cellule 4 — Q-table et hyperparamètres
n_states_raw = 11        # 0..10
n_states_sell = 11       # 0..10
n_actions = 22           # 0..21

Q = np.zeros((n_states_raw, n_states_sell, n_actions))

alpha = 0.1
gamma = 0.95
epsilon = 1.0
epsilon_min = 0.1
epsilon_decay = 0.995


In [11]:
# Cellule 5 — Politique epsilon-greedy
def choose_action(state, epsilon):
    sr, ss = state
    if random.random() < epsilon:
        return random.randint(0, n_actions - 1)
    return int(np.argmax(Q[sr, ss, :]))


In [13]:
# Cellule 6 — Boucle d’apprentissage
n_episodes = 8000
rewards_per_episode = []

for episode in range(n_episodes):
    state = env.reset()
    done = False
    total_reward = 0

    while not done:
        action = choose_action(state, epsilon)
        next_state, reward, done, _ = env.step(action)

        sr, ss = state
        nsr, nss = next_state

        best_next = np.max(Q[nsr, nss, :])
        Q[sr, ss, action] += alpha * (reward + gamma * best_next - Q[sr, ss, action])

        state = next_state
        total_reward += reward

    rewards_per_episode.append(total_reward)

    if epsilon > epsilon_min:
        epsilon *= epsilon_decay

    if (episode+1) % 200 == 0:
        print(f"Episode {episode+1}, Reward={total_reward:.1f}, epsilon={epsilon:.3f}")


Episode 200, Reward=18.5, epsilon=0.367
Episode 400, Reward=96.0, epsilon=0.135
Episode 600, Reward=80.0, epsilon=0.100
Episode 800, Reward=43.5, epsilon=0.100
Episode 1000, Reward=174.0, epsilon=0.100
Episode 1200, Reward=130.5, epsilon=0.100
Episode 1400, Reward=151.5, epsilon=0.100
Episode 1600, Reward=142.0, epsilon=0.100
Episode 1800, Reward=18.0, epsilon=0.100
Episode 2000, Reward=94.0, epsilon=0.100
Episode 2200, Reward=30.5, epsilon=0.100
Episode 2400, Reward=76.5, epsilon=0.100
Episode 2600, Reward=144.5, epsilon=0.100
Episode 2800, Reward=58.0, epsilon=0.100
Episode 3000, Reward=149.5, epsilon=0.100
Episode 3200, Reward=93.5, epsilon=0.100
Episode 3400, Reward=168.0, epsilon=0.100
Episode 3600, Reward=119.5, epsilon=0.100
Episode 3800, Reward=150.0, epsilon=0.100
Episode 4000, Reward=70.5, epsilon=0.100
Episode 4200, Reward=117.0, epsilon=0.100
Episode 4400, Reward=132.5, epsilon=0.100
Episode 4600, Reward=139.0, epsilon=0.100
Episode 4800, Reward=166.5, epsilon=0.100
Episode

In [19]:
# Cellule 7 — Affichage lisible par stock_sell

def short_action(a):
    if a == 0:
        return "Attendre"
    if 1 <= a <= 10:
        return f"P1-{a}"
    if 11 <= a <= 20:
        return f"P2-{a-10}"
    if a == 21:
        return "Commander"
    return "?"

print("=== Politique optimale (affichage par stock_sell) ===\n")

for ss in range(11):
    print(f"--- stock_sell = {ss} ---")
    print("stock_raw | action optimale")
    print("---------------------------")
    for sr in range(11):
        best = np.argmax(Q[sr, ss, :])
        print(f"{sr:<9} | {short_action(best)}")
    print()


=== Politique optimale (affichage par stock_sell) ===

--- stock_sell = 0 ---
stock_raw | action optimale
---------------------------
0         | Commander
1         | Commander
2         | P2-1
3         | P1-2
4         | P2-2
5         | P2-1
6         | P2-2
7         | P2-2
8         | P2-4
9         | P2-3
10        | P2-3

--- stock_sell = 1 ---
stock_raw | action optimale
---------------------------
0         | Commander
1         | P1-1
2         | P1-1
3         | P2-1
4         | P1-1
5         | P1-1
6         | P1-1
7         | P1-7
8         | P1-3
9         | P2-3
10        | P2-3

--- stock_sell = 2 ---
stock_raw | action optimale
---------------------------
0         | Commander
1         | Commander
2         | P1-1
3         | P2-1
4         | P1-2
5         | P1-1
6         | P2-1
7         | P2-2
8         | P1-1
9         | P1-7
10        | P1-1

--- stock_sell = 3 ---
stock_raw | action optimale
---------------------------
0         | P1-2
1         | P1-1
2     

In [21]:
# Cellule 8 — Synthèse métier intelligente 

import numpy as np
from collections import defaultdict

# Classification des comportements
zones = {
    "Commander": [],
    "Attendre": [],
    "P1": [],
    "P2": []
}

# Analyse de la politique optimale
for sr in range(11):
    for ss in range(11):
        best = np.argmax(Q[sr, ss, :])

        if best == 21:
            zones["Commander"].append((sr, ss))
        elif best == 0:
            zones["Attendre"].append((sr, ss))
        elif 1 <= best <= 10:
            zones["P1"].append((sr, ss))
        elif 11 <= best <= 20:
            zones["P2"].append((sr, ss))

# Construction d'une synthèse claire
def format_states(lst):
    """Affiche les états sous forme compacte, ex: (sr=2, ss=0), (sr=3, ss=1)..."""
    return ", ".join([f"(sr={s[0]}, ss={s[1]})" for s in lst[:20]]) + \
           ("..." if len(lst) > 20 else "")

print("\n=== Synthèse métier automatique ===\n")

print("1. Zones où l'agent choisit de COMMANDER (action 21)")
print("   → Décisions:", len(zones["Commander"]))
print("   → États concernés:", format_states(zones["Commander"]), "\n")

print("2. Zones où l'agent choisit d'ATTENDRE (action 0)")
print("   → Décisions:", len(zones["Attendre"]))
print("   → États concernés:", format_states(zones["Attendre"]), "\n")

print("3. Zones où l'agent produit du PRODUIT 1 (actions 1..10)")
print("   → Décisions:", len(zones["P1"]))
print("   → États concernés:", format_states(zones["P1"]), "\n")

print("4. Zones où l'agent produit du PRODUIT 2 (actions 11..20)")
print("   → Décisions:", len(zones["P2"]))
print("   → États concernés:", format_states(zones["P2"]), "\n")

# Résumé métier global basé sur la structure observée
print("=== Interprétation automatique (résumé métier) ===\n")

# Commande MP
if all(sr == 0 for sr, ss in zones["Commander"]):
    print("• L'agent ne commande que lorsque le stock de MP est à 0 → politique rationnelle. ")
else:
    print("• L'agent commande dans des situations inattendues → à vérifier.")

# P2 dominance
if len(zones["P2"]) > len(zones["P1"]):
    print("• Le modèle privilégie massivement le Produit 2 lorsque le stock_sell est bas : logique de forte rentabilité.")
else:
    print("• Le modèle privilégie le Produit 1 : logique prudente dictée par les coûts de stockage.")

# P1 usage
print("• Le Produit 1 est utilisé comme régulateur : petites productions lorsque stock_sell est élevé.")

# Attente
if len(zones["Attendre"]) > 0:
    print("• L'action 'Attendre' apparaît dans les zones de stock_sell élevé : éviter le surstock et la pénalité.")
else:
    print("• Aucune attente détectée → le coût de stockage est trop faible pour générer une prudence.")

print("\n=== Fin de la synthèse ===\n")



=== Synthèse métier automatique ===

1. Zones où l'agent choisit de COMMANDER (action 21)
   → Décisions: 14
   → États concernés: (sr=0, ss=0), (sr=0, ss=1), (sr=0, ss=2), (sr=0, ss=4), (sr=0, ss=5), (sr=0, ss=6), (sr=0, ss=7), (sr=0, ss=8), (sr=0, ss=9), (sr=0, ss=10), (sr=1, ss=0), (sr=1, ss=2), (sr=1, ss=4), (sr=1, ss=6) 

2. Zones où l'agent choisit d'ATTENDRE (action 0)
   → Décisions: 9
   → États concernés: (sr=7, ss=5), (sr=7, ss=7), (sr=8, ss=4), (sr=8, ss=5), (sr=8, ss=9), (sr=9, ss=4), (sr=9, ss=9), (sr=10, ss=3), (sr=10, ss=8) 

3. Zones où l'agent produit du PRODUIT 1 (actions 1..10)
   → Décisions: 64
   → États concernés: (sr=0, ss=3), (sr=1, ss=1), (sr=1, ss=3), (sr=1, ss=5), (sr=1, ss=7), (sr=1, ss=8), (sr=1, ss=9), (sr=1, ss=10), (sr=2, ss=1), (sr=2, ss=2), (sr=2, ss=3), (sr=2, ss=9), (sr=3, ss=0), (sr=3, ss=4), (sr=3, ss=5), (sr=3, ss=6), (sr=3, ss=7), (sr=3, ss=8), (sr=3, ss=9), (sr=4, ss=1)... 

4. Zones où l'agent produit du PRODUIT 2 (actions 11..20)
   → Décis